# Trajectory
here we are going to add the black box to each image with is randomly placed and move it from the left to right, specify the direction to it and the speed slow, fast medium

In [1]:
import sys
import numpy as np
import os
import os.path as op
import re
import pandas as pd
import imageio
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score
import datetime
from PIL import Image
from itertools import chain
import csv
import random
from skimage.io import imread, imsave
from skimage.transform import resize
import math

import warnings;
warnings.filterwarnings('ignore');

plt.style.use('ggplot')
%matplotlib inline

In [2]:
#orginal data anf the modifeid data set
TR_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/Training'
TR_OUT_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTraining_medtraj1'

tr_modDataExists = op.exists(TR_OUT_DATA_DIR)
    
if not tr_modDataExists:
    os.makedirs(TR_OUT_DATA_DIR) 
    
tr_maxCountPerCategory = 50

In [3]:
#testing dataset
TE_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/Test'
TE_OUT_DATA_DIR = 'C:/Users/sonali/Desktop/DS/ml/project_final/fruits/fruits-360/modTest_medtraj1'

te_modDataExists = op.exists(TE_OUT_DATA_DIR)
    
if not te_modDataExists:
    os.makedirs(TE_OUT_DATA_DIR) 
    
te_maxCountPerCategory = 10

 generate the black box and move it randomly on the image having random size 

In [4]:
def overlap1d(ipos,bpos,isize,bsize):
    iepos = ipos+isize-1
    bepos = bpos+bsize-1
    A = bpos-ipos
    B = bepos-iepos
    if A < 0:
        A = 0
    elif A > isize:
        A = isize
    if B > 0:
        B = 0
    elif B < -1 * isize:
        B = -1 * isize
    olap = isize - (A-B)   
    return olap

here we  have to generate random image size X-axis=ix and y-axis=iy, width= iw and height= ih <br>
and the black box sixe xaxis=bx, yaxis=by, width=bw and height= bh <br>
we can stor this vlaues as label data in one folder called aa-label.csv

In [5]:
def createModData(myDataDir, myModDataDir, maxCountPerCategory):
    scaleChoices = [1.5,1.75,2,2.5]
    csvData = [['Image','label','overlap','ix','iy','iw','ih','bx','by','bw','bh']]
    count = 0
    foldernames = os.listdir(myDataDir)
    for folder in foldernames:
        filenames = os.listdir(myDataDir+"/"+folder)
        countPerCategory = 0
        for filename in filenames:
            fileprefix = filename.split(".")
            if len(fileprefix) == 2:
                if fileprefix[1] == "jpg":
                    out_img_name_prefix = folder+"_"+fileprefix[0]+"_f"
                    # randomly decide size..
                    myScale = scaleChoices[random.randrange(0,len(scaleChoices))]
                    myXSize = int(100 / myScale)
                    myYSize = int(100 / myScale)
                    myXpos = random.randrange(0,100-myXSize)
                    myYpos = random.randrange(0,100-myYSize)
                    myNewImg = np.full((100,100, 3), 1.0)
                    img_n = imread(myDataDir+"/"+folder+"/"+filename)
                    img_scaled = resize(img_n,(myYSize,myXSize))
                    myNewImg[myYpos:myYpos+myYSize,myXpos:myXpos+myXSize,:] = img_scaled[:,:,:]
                    minBoxXSize = int(100 / 20)
                    minBoxYSize = int(100 / 20)
                    h = random.randrange(5,15)*minBoxYSize
                    w = random.randrange(5,15)*minBoxXSize
                    myAngle = random.randrange(-45,45,5)
                    # create frames.. 
                    #frameStep = 10
                    frameStep = random.randrange(8,14)
                    for n in range(0,int(100/frameStep)):
                        img_n = np.copy(myNewImg)
                        
                        # Generate box co-ordinates
                        bx = n*frameStep
                        by = int(50 - ((bx-50)*math.tan(math.radians(myAngle))))
                        modw = min(w,100-bx)
                        modh = min(h,100-by)
                        myBox = np.full((modh,modw, 3), 0.0)
                        #print(myAngle,bx,by,modw,modh)
                        img_n[by:by+modh,bx:bx+modw,:] = myBox[:,:,:]
                        imsave(myModDataDir+"/"+out_img_name_prefix+str(n)+".jpg",img_n)
                        
                        # Finding overlap
                        X_overlap = overlap1d(myXpos,bx,myXSize,modw)
                        Y_overlap = overlap1d(myYpos,by,myYSize,modh)
                        total_overlap = int(((X_overlap * Y_overlap)/(myXSize*myYSize))*100)
                        levels = 5
                        split = 0
                        for i in range(1,levels):
                            if total_overlap > int(100/2**i):
                                split = i
                                break
                        if split == 0:
                            split = levels
                        total_overlap_round = levels - split
                        #gran = 100/levels
                        #offset = gran/2
                        #total_overlap_round = int((total_overlap+offset)/gran)
                        #total_overlap_round = round(total_overlap,1)*10
                        csvData.append([out_img_name_prefix+str(n),total_overlap_round,total_overlap,myXpos,myYpos,myXSize,myYSize,bx,by,modw,modh])
                    if count % 100 == 0:
                        print(count,": saved img - ", folder,"/",filename)
                    count = count + 1
                    countPerCategory = countPerCategory + 1 
                else:
                    print("Ignored (extension not jpg) file: ", folder,"/",filename)
            else:
                print("Ignored (undefined file prefix) file: ", folder,"/",filename)
            if countPerCategory == maxCountPerCategory:
                break
    with open(myModDataDir+"/aa-labels.csv", 'w') as csvFile:
        writer = csv.writer(csvFile)
        for line in csvData:
            writer.writerow(line)
    csvFile.close()

In [6]:
createModData(TR_DATA_DIR, TR_OUT_DATA_DIR, tr_maxCountPerCategory)

0 : saved img -  Apple Braeburn / 0_100.jpg
100 : saved img -  Apple Golden 2 / 0_100.jpg
200 : saved img -  Apple Granny Smith / 0_100.jpg
300 : saved img -  Apple Red 2 / 0_100.jpg
400 : saved img -  Apple Red Delicious / 100_100.jpg
500 : saved img -  Apple Red Yellow 2 / 0_100.jpg
600 : saved img -  Avocado / 0_100.jpg
700 : saved img -  Banana / 0_100.jpg
800 : saved img -  Banana Red / 0_100.jpg
900 : saved img -  Cantaloupe 1 / 0_100.jpg
1000 : saved img -  Carambula / 0_100.jpg
1100 : saved img -  Cherry 2 / 105_100.jpg
1200 : saved img -  Cherry Wax Black / 0_100.jpg
1300 : saved img -  Cherry Wax Yellow / 0_100.jpg
1400 : saved img -  Clementine / 10_100.jpg
1500 : saved img -  Dates / 0_100.jpg
1600 : saved img -  Grape Blue / 0_100.jpg
1700 : saved img -  Grape White / 0_100.jpg
1800 : saved img -  Grape White 3 / 0_100.jpg
1900 : saved img -  Grapefruit Pink / 0_100.jpg
2000 : saved img -  Guava / 0_100.jpg
2100 : saved img -  Huckleberry / 104_100.jpg
2200 : saved img -  

In [7]:
createModData(TE_DATA_DIR, TE_OUT_DATA_DIR, te_maxCountPerCategory)

0 : saved img -  Apple Braeburn / 321_100.jpg
100 : saved img -  Apple Red Yellow 2 / 106_100.jpg
200 : saved img -  Carambula / 101_100.jpg
300 : saved img -  Dates / 100_100.jpg
400 : saved img -  Guava / 100_100.jpg
500 : saved img -  Mandarine / 147_100.jpg
600 : saved img -  Peach / 321_100.jpg
700 : saved img -  Physalis with Husk / 11_100.jpg
800 : saved img -  Rambutan / 128_100.jpg
900 : saved img -  Tomato 3 / 0_100.jpg
